In [5]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

In [26]:
url = "https://www.pgnmentor.com/files.html#openings"

res = req.get(url)

soup = bs(res.content, 'html.parser')
rows = soup.find_all('tr')
piece_map = {
    "Knight": " N",
    "Bishop": " B",
    "King": " K",
    "Queen": " Q",
    "Rook": " R"
}
data = {"Opening": [], "Notation": []}
for row in rows:
    td_tags = row.find_all('td', style="background-image: url('txtbg.gif')")

    if len(td_tags) >= 3:
        opening_info = td_tags[1].get_text(separator=" ").strip()
        
        opening_info= opening_info.split(" ")
        opening_info.pop(-1)
        opening_info.pop(-1)
        opening_info = " ".join(opening_info)
        notation_td = td_tags[2]
        notation = ""
        for content in notation_td.contents:
            if content.name == "img":  
                piece = content.get("alt", "")  
                notation += piece_map.get(piece, "")  
            elif isinstance(content, str):  
                notation += content.strip()

        data["Notation"].append(notation)
        data["Opening"].append(opening_info)


Trompowsky, Other
['1.d4 Nf6 2. Bg5 d5', ' c5', ' g6']
Dutch, Other
['1.d4 f5 2. Nc3', ' Bg5', ' e4']
Grunfeld, Other
['1.d4 Nf6 2.c4 g6 3. Nc3 d5 4. Bf4', ' Bg5', ' e3', ' Qb3']
Queen's Indian, Other
['1.d4 Nf6 2.c4 e6 3. Nf3 b6 4. Bg5', ' Bf4']
Queen's Indian, 4.g3 Other
['1.d4 Nf6 2.c4 e6 3. Nf3 b6 4.g3 Bb7', ' Bb4+']
Nimzo-Indian, Other 4th moves
['1.d4 Nf6 2.c4 e6 3. Nc3 Bb4 4.g3', ' Qb3', ' Bd2']
Nimzo-Indian, Classical, Other
['1.d4 Nf6 2.c4 e6 3. Nc3 Bb4 4. Qc2 d5', ' Nc6', ' b6', ' d6']
Nimzo-Indian, Rubinstein, Other
['1.d4 Nf6 2.c4 e6 3. Nc3 Bb4 4.e3 b6', ' d5', ' Nc6', ' Bxc3+']
King's Indian, Classical Other 7th moves
['1.d4 Nf6 2.c4 g6 3. Nc3 Bg7 4.e4 d6 5. Nf3 O-O 6. Be2 e5 7. Be3', ' dxe5', ' Bg51.d4 Nf6 2.c4 g6 3. Nc3 Bg7 4.e4 d6 5. Nf3 O-O 6. Be2 e5 7.O-O Nbd7', ' Na6', ' exd4', ' c6', ' Nh5', ' Qe8']
King's Indian, Classical Other 5th and 6th moves
['1.d4 Nf6 2.c4 g6 3. Nc3 Bg7 4.e4 d6 5. Nf3 Bg4', ' Nbd7', ' c6', ' c51.d4 Nf6 2.c4 g6 3. Nc3 Bg7 4.e4 d6 5. Nf3 O-O 6.

In [46]:
df = pd.DataFrame.from_dict(data)
df.to_csv("openings.csv")

NameError: name 'data' is not defined

In [6]:
df=pd.read_csv("openings.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)
old_records = df.to_dict(orient="records")
new_records = []
i = "1.d4 Nf6 2.c4 g6 3. Nc3 Bg7 4.e4 d6 5. Nf3 O-O 6. Be2 e5 7. Be3, dxe5, Bg5, 1.d4 Nf6 2.c4 g6 3. Nc3 Bg7 4.e4 d6 5. Nf3 O-O 6. Be2 e5 7.O-O Nbd7, Na6, exd4, c6, Nh5, Qe8"
def fix_notation(record: dict)-> None:
    notation = record["Notation"]
    opening = record["Opening"]
    idx = 0 
    notation = notation.replace(". ", ".")
    arr = notation.split(", ")

    while idx < len(arr):
        new_records.append({"Opening": opening,"Notation": arr[idx]})
        moves = re.split(r'\d+\.', arr[idx])[1:]
        j = idx+1
        while j < len(arr) and not arr[j][0].isdigit():
            last = moves[-1].split(" ")
            last[-1]= arr[j]
            moves[-1] = " ".join(last)
            notacion = ""
            for i in range(len(moves)):
                notacion += f"{i+1}.{moves[i]}"
            new_records.append({"Opening": opening,"Notation": notacion})

            j+=1
        idx=j

[fix_notation(i) for i in old_records]
df = pd.DataFrame.from_records(new_records)
df.to_csv("fixed_openings.csv")

In [ ]:
import chess.pgn
import io

for record in new_records:
    pgn = io.StringIO(record["Notation"])
    #print(record["Opening"])
    game = chess.pgn.read_game(pgn)
    if game.errors:
        print(game.errors)
        print(record["Opening"])
        print(record["notation"])

illegal san: 'd4' in rnbqkbnr/pp2pppp/8/2pp4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 3 while parsing <Game at 0x10f16e210 ('?' vs. '?', '????.??.??' at '?')>


[IllegalMoveError("illegal san: 'd4' in rnbqkbnr/pp2pppp/8/2pp4/2PP4/8/PP2PPPP/RNBQKBNR w KQkq - 0 3")]
